# Tekstclassificatietaak

In deze module beginnen we met een eenvoudige tekstclassificatietaak gebaseerd op de **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** dataset: we classificeren nieuwsheadlines in een van de 4 categorieën: Wereld, Sport, Zakelijk en Wetenschap/Technologie.

## De Dataset

Om de dataset te laden, gebruiken we de **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

We kunnen nu toegang krijgen tot de trainings- en testgedeelten van de dataset door respectievelijk `dataset['train']` en `dataset['test']` te gebruiken:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Laten we de eerste 10 nieuwe koppen uit onze dataset afdrukken:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Tekstvectorisatie

Nu moeten we tekst omzetten in **nummers** die kunnen worden weergegeven als tensors. Als we een woordniveau-representatie willen, moeten we twee dingen doen:

* Gebruik een **tokenizer** om tekst op te splitsen in **tokens**.
* Bouw een **vocabulaire** van die tokens.

### Beperken van de vocabulairegrootte

In het voorbeeld van de AG News-dataset is de vocabulairegrootte behoorlijk groot, meer dan 100k woorden. Over het algemeen hebben we geen woorden nodig die zelden in de tekst voorkomen — slechts een paar zinnen zullen ze bevatten, en het model zal er niet van leren. Daarom is het logisch om de vocabulairegrootte te beperken tot een kleiner aantal door een argument door te geven aan de vectorizer-constructor:

Beide stappen kunnen worden afgehandeld met behulp van de **TextVectorization**-laag. Laten we het vectorizer-object instantiëren en vervolgens de `adapt`-methode aanroepen om alle tekst door te nemen en een vocabulaire op te bouwen:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Let op** dat we slechts een deel van de volledige dataset gebruiken om een vocabulaire op te bouwen. We doen dit om de uitvoeringstijd te versnellen en u niet te laten wachten. Echter, we lopen het risico dat sommige woorden uit de volledige dataset niet in de vocabulaire worden opgenomen en tijdens de training worden genegeerd. Het gebruik van de volledige vocabulaire en het doorlopen van de hele dataset tijdens `adapt` zou de uiteindelijke nauwkeurigheid moeten verhogen, maar niet significant.

Nu kunnen we toegang krijgen tot de daadwerkelijke vocabulaire:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Met behulp van de vectorizer kunnen we eenvoudig elke tekst coderen in een reeks getallen:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Bag-of-words tekstrepresentatie

Omdat woorden betekenis vertegenwoordigen, kunnen we soms de betekenis van een stuk tekst achterhalen door alleen naar de afzonderlijke woorden te kijken, ongeacht hun volgorde in de zin. Bijvoorbeeld, bij het classificeren van nieuws, zullen woorden zoals *weer* en *sneeuw* waarschijnlijk wijzen op *weersvoorspellingen*, terwijl woorden zoals *aandelen* en *dollar* eerder zouden duiden op *financieel nieuws*.

**Bag-of-words** (BoW) vectorrepresentatie is de meest eenvoudige traditionele vectorrepresentatie om te begrijpen. Elk woord is gekoppeld aan een vectorindex, en een element in de vector bevat het aantal keren dat elk woord voorkomt in een bepaald document.

![Afbeelding die laat zien hoe een bag-of-words vectorrepresentatie in het geheugen wordt weergegeven.](../../../../../translated_images/nl/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Je kunt BoW ook zien als de som van alle one-hot-gecodeerde vectoren voor individuele woorden in de tekst.

Hieronder staat een voorbeeld van hoe je een bag-of-words representatie kunt genereren met behulp van de Scikit Learn python bibliotheek:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

We kunnen ook de Keras-vectorizer gebruiken die we hierboven hebben gedefinieerd, waarbij elk woordnummer wordt omgezet in een one-hot encoding en al die vectoren worden opgeteld:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Opmerking**: Het kan je verrassen dat het resultaat verschilt van het vorige voorbeeld. De reden hiervoor is dat in het Keras-voorbeeld de lengte van de vector overeenkomt met de grootte van de woordenschat, die is opgebouwd uit de hele AG News-dataset, terwijl we in het Scikit Learn-voorbeeld de woordenschat ter plekke hebben opgebouwd uit de voorbeeldtekst.


## Het trainen van de BoW-classificator

Nu we hebben geleerd hoe we de bag-of-words-representatie van onze tekst kunnen maken, laten we een classificator trainen die hiervan gebruik maakt. Eerst moeten we onze dataset omzetten naar een bag-of-words-representatie. Dit kan worden gedaan met behulp van de `map`-functie op de volgende manier:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Laten we nu een eenvoudige classifier-neuraal netwerk definiëren dat één lineaire laag bevat. De invoergrootte is `vocab_size`, en de uitvoergrootte komt overeen met het aantal klassen (4). Omdat we een classificatietaak oplossen, is de laatste activatiefunctie **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Aangezien we 4 klassen hebben, is een nauwkeurigheid van boven de 80% een goed resultaat.

## Een classifier trainen als één netwerk

Omdat de vectorizer ook een Keras-laag is, kunnen we een netwerk definiëren dat deze bevat en het end-to-end trainen. Op deze manier hoeven we de dataset niet te vectoriseren met behulp van `map`, we kunnen gewoon de originele dataset doorgeven aan de input van het netwerk.

> **Opmerking**: We zouden nog steeds maps moeten toepassen op onze dataset om velden uit woordenboeken (zoals `title`, `description` en `label`) om te zetten naar tuples. Echter, bij het laden van data vanaf schijf, kunnen we vanaf het begin een dataset bouwen met de vereiste structuur.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrams, trigrams en n-grams

Een beperking van de bag-of-words aanpak is dat sommige woorden deel uitmaken van meerwoordige uitdrukkingen. Bijvoorbeeld, het woord 'hot dog' heeft een compleet andere betekenis dan de woorden 'hot' en 'dog' in andere contexten. Als we de woorden 'hot' en 'dog' altijd met dezelfde vectoren representeren, kan dat ons model in verwarring brengen.

Om dit aan te pakken, worden **n-gram representaties** vaak gebruikt bij methoden voor documentclassificatie, waarbij de frequentie van elk woord, twee-woordcombinatie of drie-woordcombinatie een nuttige eigenschap is voor het trainen van classifiers. Bij bigram representaties voegen we bijvoorbeeld alle woordparen toe aan de woordenschat, naast de oorspronkelijke woorden.

Hieronder staat een voorbeeld van hoe je een bigram bag-of-words representatie kunt genereren met behulp van Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Het grootste nadeel van de n-gram aanpak is dat de omvang van de woordenschat extreem snel begint te groeien. In de praktijk moeten we de n-gram representatie combineren met een techniek voor dimensiereductie, zoals *embeddings*, waar we in de volgende eenheid op ingaan.

Om een n-gram representatie te gebruiken in ons **AG News** dataset, moeten we de parameter `ngrams` doorgeven aan onze `TextVectorization` constructor. De omvang van een bigram woordenschat is **aanzienlijk groter**, in ons geval meer dan 1,3 miljoen tokens! Daarom is het logisch om ook het aantal bigram tokens te beperken tot een redelijk aantal.

We zouden dezelfde code als hierboven kunnen gebruiken om de classifier te trainen, maar dat zou zeer inefficiënt zijn qua geheugen. In de volgende eenheid zullen we de bigram classifier trainen met behulp van embeddings. In de tussentijd kun je experimenteren met het trainen van de bigram classifier in dit notebook en kijken of je een hogere nauwkeurigheid kunt behalen.


## Automatisch BoW-vectoren berekenen

In het bovenstaande voorbeeld hebben we BoW-vectoren met de hand berekend door de één-op-één encoderingen van individuele woorden op te tellen. De nieuwste versie van TensorFlow stelt ons echter in staat om BoW-vectoren automatisch te berekenen door de parameter `output_mode='count` door te geven aan de vectorizer constructor. Dit maakt het definiëren en trainen van ons model aanzienlijk eenvoudiger:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Termfrequentie - inverse documentfrequentie (TF-IDF)

In de BoW-representatie worden woordvoorkomens op dezelfde manier gewogen, ongeacht het woord zelf. Het is echter duidelijk dat veelvoorkomende woorden zoals *een* en *in* veel minder belangrijk zijn voor classificatie dan gespecialiseerde termen. Bij de meeste NLP-taken zijn sommige woorden relevanter dan andere.

**TF-IDF** staat voor **termfrequentie - inverse documentfrequentie**. Het is een variatie op bag-of-words, waarbij in plaats van een binaire 0/1-waarde die aangeeft of een woord in een document voorkomt, een drijvende-kommawaarde wordt gebruikt die gerelateerd is aan de frequentie van het woord in de corpus.

Meer formeel wordt het gewicht $w_{ij}$ van een woord $i$ in document $j$ gedefinieerd als:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
waarbij
* $tf_{ij}$ het aantal voorkomens van $i$ in $j$ is, oftewel de BoW-waarde die we eerder hebben gezien
* $N$ het aantal documenten in de collectie is
* $df_i$ het aantal documenten is waarin het woord $i$ voorkomt in de hele collectie

De TF-IDF-waarde $w_{ij}$ neemt proportioneel toe met het aantal keren dat een woord in een document voorkomt en wordt gecorrigeerd door het aantal documenten in de corpus waarin het woord voorkomt. Dit helpt om te compenseren voor het feit dat sommige woorden vaker voorkomen dan andere. Bijvoorbeeld, als het woord in *elk* document in de collectie voorkomt, geldt $df_i=N$, en $w_{ij}=0$, en die termen worden volledig genegeerd.

Je kunt eenvoudig een TF-IDF-vectorisatie van tekst maken met Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In Keras kan de `TextVectorization`-laag automatisch TF-IDF-frequenties berekenen door de parameter `output_mode='tf-idf'` door te geven. Laten we de code die we hierboven hebben gebruikt herhalen om te zien of het gebruik van TF-IDF de nauwkeurigheid verhoogt:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Conclusie

Hoewel TF-IDF-representaties frequentiegewichten toekennen aan verschillende woorden, zijn ze niet in staat om betekenis of volgorde weer te geven. Zoals de beroemde taalkundige J. R. Firth in 1935 zei: "De volledige betekenis van een woord is altijd contextueel, en geen enkele studie van betekenis los van context kan serieus worden genomen." Later in de cursus zullen we leren hoe we contextuele informatie uit tekst kunnen vastleggen met behulp van taalmodellen.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
